
# 🌱 Predicting Biofuel Production from Agricultural Waste

This notebook demonstrates a **simple AI pipeline** to estimate biofuel energy potential from crop production data.

Steps:
1. Download a dataset from **Kaggle** (Crop Production in India – 1997–2021).
2. Preprocess the dataset (clean columns, handle missing values).
3. Compute **Residue-to-Product Ratio (RPR)** × **Calorific Value (CV)** → Energy (GJ).
4. Train a **RandomForest Regressor** to predict biofuel potential.
5. Evaluate the model.

> ⚠️ Make sure you have a Kaggle API token (`kaggle.json`) placed in the correct folder before running.


In [6]:

# Install Kaggle API if not already installed
#!pip install -q kaggle

#import os, zipfile

# Dataset: India Crop Production (1997–2021)
#DATASET = "mohansachdeva/india-crop-production-1997-2021"

#os.makedirs("data", exist_ok=True)
#!kaggle datasets download -d {DATASET} -p data

# Unzip files
#for fn in os.listdir("data"):
   # if fn.endswith(".zip"):
        with zipfile.ZipFile(os.path.join("data", fn), "r") as z:
            z.extractall("data")
        os.remove(os.path.join("data", fn))

#os.listdir("data")


IndentationError: unexpected indent (1940256758.py, line 15)

In [7]:

import pandas as pd
import numpy as np

# Load dataset (pick first CSV file)
#files = [f for f in os.listdir("data") if f.endswith(".csv")]
df = pd.read_csv(r"C:\Users\shrut\OneDrive\Desktop\abc\archive (3)\crop_production.csv")
df


,State_Name,District_Name,Crop_Year,Season,Crop,Area,Production
0,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Arecanut,1254.0,2000.0
1,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Other Kharif pulses,2.0,1.0
2,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Rice,102.0,321.0
3,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Banana,176.0,641.0
4,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Cashewnut,720.0,165.0
...,...,...,...,...,...,...,...
246086,West Bengal,PURULIA,2014,Summer,Rice,306.0,801.0
246087,West Bengal,PURULIA,2014,Summer,Sesamum,627.0,463.0
246088,West Bengal,PURULIA,2014,Whole Year,Sugarcane,324.0,16250.0
246089,West Bengal,PURULIA,2014,Winter,Rice,279151.0,597899.0


In [8]:
df.head()

,State_Name,District_Name,Crop_Year,Season,Crop,Area,Production
0,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Arecanut,1254.0,2000.0
1,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Other Kharif pulses,2.0,1.0
2,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Rice,102.0,321.0
3,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Banana,176.0,641.0
4,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Cashewnut,720.0,165.0


In [9]:

# Standardize column names
df.columns = [c.strip().replace(" ", "_") for c in df.columns]

rename_map = {
    "state_name": "State_Name",
    "district_name": "District_Name",
    "season": "Season",
    "crop": "Crop",
    "area": "Area",
    "production": "Production"
}
df = df.rename(columns={k:v for k,v in rename_map.items() if k in df.columns.str.lower().tolist()})

needed = ["State_Name","District_Name","Season","Crop","Area","Production"]
df = df[[c for c in df.columns if c in needed]].copy()

# Handle numeric
df["Area"] = pd.to_numeric(df["Area"], errors="coerce")
df["Production"] = pd.to_numeric(df["Production"], errors="coerce")
df = df.dropna(subset=["Crop","Area","Production"]).reset_index(drop=True)

print(df.shape)
df.head()


(242361, 6)


,State_Name,District_Name,Season,Crop,Area,Production
0,Andaman and Nicobar Islands,NICOBARS,Kharif,Arecanut,1254.0,2000.0
1,Andaman and Nicobar Islands,NICOBARS,Kharif,Other Kharif pulses,2.0,1.0
2,Andaman and Nicobar Islands,NICOBARS,Kharif,Rice,102.0,321.0
3,Andaman and Nicobar Islands,NICOBARS,Whole Year,Banana,176.0,641.0
4,Andaman and Nicobar Islands,NICOBARS,Whole Year,Cashewnut,720.0,165.0


In [10]:

# Residue-to-Product Ratio (RPR) and Calorific Value (MJ/kg)
RPR_CV = {
    "Sugarcane": {"rpr": 0.28, "cv": 18.5},
    "Rice": {"rpr": 0.22, "cv": 14.5},
    "Wheat": {"rpr": 1.40, "cv": 17.5},
    "Maize": {"rpr": 1.00, "cv": 18.0},
    "Cotton": {"rpr": 2.00, "cv": 16.5},
    "Soybean": {"rpr": 1.50, "cv": 17.2},
}

def compute_energy(row):
    crop = str(row["Crop"]).strip().title()
    crop = {"Paddy":"Rice","Soyabean":"Soybean"}.get(crop, crop)
    rpr = RPR_CV.get(crop, {}).get("rpr", 1.0)
    cv = RPR_CV.get(crop, {}).get("cv", 17.0)
    residue_tonnes = row["Production"] * rpr
    energy_gj = residue_tonnes * 1000 * cv / 1000  # tonnes→kg, MJ/kg → MJ, /1000 → GJ
    return residue_tonnes, energy_gj

df[["Residue_tonnes","Energy_GJ"]] = df.apply(lambda r: compute_energy(r), axis=1, result_type="expand")
df.head()


,State_Name,District_Name,Season,Crop,Area,Production,Residue_tonnes,Energy_GJ
0,Andaman and Nicobar Islands,NICOBARS,Kharif,Arecanut,1254.0,2000.0,2000.00,34000.00
1,Andaman and Nicobar Islands,NICOBARS,Kharif,Other Kharif pulses,2.0,1.0,1.00,17.00
2,Andaman and Nicobar Islands,NICOBARS,Kharif,Rice,102.0,321.0,70.62,1023.99
3,Andaman and Nicobar Islands,NICOBARS,Whole Year,Banana,176.0,641.0,641.00,10897.00
4,Andaman and Nicobar Islands,NICOBARS,Whole Year,Cashewnut,720.0,165.0,165.00,2805.00


In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

# Features and target
target = "Energy_GJ"
y = df[target]

cat_cols = ["State_Name","District_Name","Season","Crop"]
num_cols = ["Area","Production"]
X = df[cat_cols + num_cols]

# Preprocess (encode categoricals)
pre = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
    ("num", "passthrough", num_cols)
])

# Model pipeline
pipe = Pipeline([
    ("prep", pre),
    ("rf", RandomForestRegressor(n_estimators=200, random_state=42))
])

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
pipe.fit(X_train, y_train)

# Predictions & evaluation
pred = pipe.predict(X_test)
mae = mean_absolute_error(y_test, pred)
r2 = r2_score(y_test, pred)

print("MAE (GJ):", mae)
print("R²:", r2)


In [ ]:

import joblib, os, json

os.makedirs("models", exist_ok=True)
joblib.dump(pipe, "models/biofuel_model.joblib")

meta = {
    "categorical_features": cat_cols,
    "numerical_features": num_cols,
    "target": target
}
with open("models/metadata.json", "w") as f:
    json.dump(meta, f, indent=2)

print("✅ Model and metadata saved in 'models/' folder")
